# Step 1: Download and Unzip the Dataset

In [4]:
!wget -P data https://go.criteo.net/criteo-research-attribution-dataset.zip

--2025-03-22 19:24:29--  https://go.criteo.net/criteo-research-attribution-dataset.zip
Resolving go.criteo.net (go.criteo.net)... 74.119.117.38, 2620:100:a00b::27
Connecting to go.criteo.net (go.criteo.net)|74.119.117.38|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://criteostorage.blob.core.windows.net/criteo-research-datasets/criteo_attribution_dataset.zip [following]
--2025-03-22 19:24:29--  https://criteostorage.blob.core.windows.net/criteo-research-datasets/criteo_attribution_dataset.zip
Resolving criteostorage.blob.core.windows.net (criteostorage.blob.core.windows.net)... 20.209.1.1
Connecting to criteostorage.blob.core.windows.net (criteostorage.blob.core.windows.net)|20.209.1.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 653128946 (623M) [application/zip]
Saving to: ‘data/criteo-research-attribution-dataset.zip’

criteo-research-att 100%[===================>] 622.87M  7.78MB/s    in 87s     

2025-03-22 19:25:

In [ ]:
# Check if the .zip file is in the data folder
!ls -lh data/

total 623M
-rw-rw-r-- 1 niresh niresh 623M Apr  8  2020 criteo-research-attribution-dataset.zip


In [ ]:
# Extracting the file
!unzip data/criteo-research-attribution-dataset.zip -d data/

Archive:  data/criteo-research-attribution-dataset.zip
  inflating: data/Experiments.ipynb  
  inflating: data/README.md          
  inflating: data/criteo_attribution_dataset.tsv.gz  


# 1. Loading the Dataset

In [ ]:
import pandas as pd

file_path = "data/criteo_attribution_dataset.tsv.gz"

# Read the file in chunks so it doesn't run out of memory and prevent kernel crash
chunk_size = 500_000  # Load 500K rows at a time
df_list = []

for chunk in pd.read_csv(file_path, sep="\t", compression="gzip", chunksize=chunk_size):
    df_list.append(chunk)
    print(f"Loaded {len(df_list) * chunk_size} rows so far...")

# Combine chunks into a single dataframe
df = pd.concat(df_list, ignore_index=True)

print("Data loaded successfully!")

Loaded 500000 rows so far...
Loaded 1000000 rows so far...
Loaded 1500000 rows so far...
Loaded 2000000 rows so far...
Loaded 2500000 rows so far...
Loaded 3000000 rows so far...
Loaded 3500000 rows so far...
Loaded 4000000 rows so far...
Loaded 4500000 rows so far...
Loaded 5000000 rows so far...
Loaded 5500000 rows so far...
Loaded 6000000 rows so far...
Loaded 6500000 rows so far...
Loaded 7000000 rows so far...
Loaded 7500000 rows so far...
Loaded 8000000 rows so far...
Loaded 8500000 rows so far...
Loaded 9000000 rows so far...
Loaded 9500000 rows so far...
Loaded 10000000 rows so far...
Loaded 10500000 rows so far...
Loaded 11000000 rows so far...
Loaded 11500000 rows so far...
Loaded 12000000 rows so far...
Loaded 12500000 rows so far...
Loaded 13000000 rows so far...
Loaded 13500000 rows so far...
Loaded 14000000 rows so far...
Loaded 14500000 rows so far...
Loaded 15000000 rows so far...
Loaded 15500000 rows so far...
Loaded 16000000 rows so far...
Loaded 16500000 rows so far.

In [ ]:
# Previewing the table
df.head()

,timestamp,uid,campaign,conversion,conversion_timestamp,conversion_id,attribution,click,click_pos,click_nb,...,time_since_last_click,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9
0,0,20073966,22589171,0,-1,-1,0,0,-1,-1,...,-1,5824233,9312274,3490278,29196072,11409686,1973606,25162884,29196072,29196072
1,2,24607497,884761,0,-1,-1,0,0,-1,-1,...,423858,30763035,9312274,14584482,29196072,11409686,1973606,22644417,9312274,21091111
2,2,28474333,18975823,0,-1,-1,0,0,-1,-1,...,8879,138937,9312274,10769841,29196072,5824237,138937,1795451,29196072,15351056
3,3,7306395,29427842,1,1449193,3063962,0,1,0,7,...,-1,28928366,26597095,12435261,23549932,5824237,1973606,9180723,29841067,29196072
4,3,25357769,13365547,0,-1,-1,0,0,-1,-1,...,-1,138937,26597094,31616034,29196072,11409684,26597096,4480345,29196072,29196072


**Here is a detailed description of the fields (they are tab-separated in the file):**

timestamp: timestamp of the impression (starting from 0 for the first impression). The dataset is sorted according to timestamp.

uid: a unique user identifier

campaign: a unique identifier for the campaign

conversion: 1 if there was a conversion in the 30 days after the impression (independently of whether this impression was last click or not)

conversion_timestamp: the timestamp of the conversion or -1 if no conversion was observed

conversion_id: a unique identifier for each conversion (so that timelines can be reconstructed if needed). -1 if there was no conversion

attribution: 1 if the conversion was attributed to Criteo, 0 otherwise

click: 1 if the impression was clicked, 0 otherwise

click_pos: the position of the click before a conversion (0 for first-click)

click_nb: number of clicks. More than 1 if there was several clicks before a conversion

cost: the price paid by Criteo for this display (disclaimer: not the real price, only a transformed version of it)

cpo: the cost-per-order in case of attributed conversion (disclaimer: not the real price, only a transformed version of it)

time_since_last_click: the time since the last click (in s) for the given impression

cat[1-9]: contextual features associated to the display. Can be used to learn the click/conversion models. We do not disclose the meaning of these features but it is not relevant for this study. Each column is a categorical variable. In the experiments, they are mapped to a fixed dimensionality space using the Hashing Trick (see paper for reference).

In [6]:
# Checking datatypes
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16468027 entries, 0 to 16468026
Data columns (total 22 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   timestamp              int64  
 1   uid                    int64  
 2   campaign               int64  
 3   conversion             int64  
 4   conversion_timestamp   int64  
 5   conversion_id          int64  
 6   attribution            int64  
 7   click                  int64  
 8   click_pos              int64  
 9   click_nb               int64  
 10  cost                   float64
 11  cpo                    float64
 12  time_since_last_click  int64  
 13  cat1                   int64  
 14  cat2                   int64  
 15  cat3                   int64  
 16  cat4                   int64  
 17  cat5                   int64  
 18  cat6                   int64  
 19  cat7                   int64  
 20  cat8                   int64  
 21  cat9                   int64  
dtypes: float64(2), i

**Key Insights and Train of Thought**

uid, campaign are of type int. Having them as string makes more sense.

timestamp columns can be converted to datetimestamps for more interpretability and validation.

We don't have the actual meaning of contextual features, but assuming we do, we can see how it affects out MTA models and learn more about when they convert.

In [7]:
# Before we convert datatypes, let's get an understanding of the descriptive statistics

df.describe()

: 

Challenge - Pandas isn't the best option to understand this data because of the data volume. It often runs out of memory.

Let's try using dask and duckDB as:

1️⃣ Using Dask for large-scale exploration:

Works like Pandas but scales well ✅

Handles memory efficiently ✅

Allows parallel processing ✅

2️⃣ Using DuckDB for fast SQL queries

No need to load the whole dataset ✅

Super fast for filtering and aggregation ✅

In [1]:
import dask.dataframe as dd

# Path to dataset
file_path = "data/criteo_attribution_dataset.tsv.gz"  # Update if needed

# Load dataset using Dask
df = dd.read_csv(file_path, sep="\t", compression="gzip", blocksize=None, assume_missing=True)

# Check column names without triggering full computation
print(df.columns)

Index(['timestamp', 'uid', 'campaign', 'conversion', 'conversion_timestamp',
       'conversion_id', 'attribution', 'click', 'click_pos', 'click_nb',
       'cost', 'cpo', 'time_since_last_click', 'cat1', 'cat2', 'cat3', 'cat4',
       'cat5', 'cat6', 'cat7', 'cat8', 'cat9'],
      dtype='object')


In [ ]:
# Fetch only 5 rows and compute
small_sample = df.sample(frac=0.01, random_state=42).compute()
print(small_sample)

: 

Seems like using dask as well is having trouble with memory utilization

In [1]:
import duckdb

# Trying duckdb to load everything
query = "SELECT * FROM 'data/criteo_attribution_dataset.tsv.gz' LIMIT 100000"
df = duckdb.query(query).to_df()

df.head()

,timestamp,uid,campaign,conversion,conversion_timestamp,conversion_id,attribution,click,click_pos,click_nb,...,time_since_last_click,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9
0,0,20073966,22589171,0,-1,-1,0,0,-1,-1,...,-1,5824233,9312274,3490278,29196072,11409686,1973606,25162884,29196072,29196072
1,2,24607497,884761,0,-1,-1,0,0,-1,-1,...,423858,30763035,9312274,14584482,29196072,11409686,1973606,22644417,9312274,21091111
2,2,28474333,18975823,0,-1,-1,0,0,-1,-1,...,8879,138937,9312274,10769841,29196072,5824237,138937,1795451,29196072,15351056
3,3,7306395,29427842,1,1449193,3063962,0,1,0,7,...,-1,28928366,26597095,12435261,23549932,5824237,1973606,9180723,29841067,29196072
4,3,25357769,13365547,0,-1,-1,0,0,-1,-1,...,-1,138937,26597094,31616034,29196072,11409684,26597096,4480345,29196072,29196072


In [2]:
# Storing as a DuckDB table for faster queries

# Create a persistent DuckDB database
duckdb.sql("CREATE TABLE criteo_data AS SELECT * FROM read_csv_auto('data/criteo_attribution_dataset.tsv.gz', sep='\t')")

In [4]:
duckdb.sql("""
    SELECT count(*) as row_cnt
    FROM criteo_data
""").df()


,row_cnt
0,16468027


DuckDB seems to be working

In [5]:
duckdb.sql("DESCRIBE criteo_data").df()

,column_name,column_type,null,key,default,extra
0,timestamp,BIGINT,YES,None,None,None
1,uid,BIGINT,YES,None,None,None
2,campaign,BIGINT,YES,None,None,None
3,conversion,BIGINT,YES,None,None,None
4,conversion_timestamp,BIGINT,YES,None,None,None
5,conversion_id,BIGINT,YES,None,None,None
6,attribution,BIGINT,YES,None,None,None
7,click,BIGINT,YES,None,None,None
8,click_pos,BIGINT,YES,None,None,None
9,click_nb,BIGINT,YES,None,None,None
